In [1]:
import pandas as pd
import numpy as np
import io
from google.colab import files

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['Dataset_Ads.csv']))

Saving Dataset_Ads.csv to Dataset_Ads.csv


In [2]:
TOTAL_BUDGET = 1_000_000

# Load & clean
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# Detect CTR / CR columns
possible_ctr = [c for c in df.columns if "ctr" in c.lower()]
possible_conv = [
    c for c in df.columns
    if ("conversion" in c.lower() and "rate" in c.lower()) or c.lower() == "conversion_rate"
]

ctr_col = possible_ctr[0] if possible_ctr else None
conv_col = possible_conv[0] if possible_conv else None

# Derive columns if missing
if ctr_col is None and {"clicks", "impressions"}.issubset(df.columns):
    df["ctr"] = df["clicks"] / df["impressions"].replace(0, np.nan)
    ctr_col = "ctr"

if conv_col is None and {"conversions", "clicks"}.issubset(df.columns):
    df["conversion_rate"] = df["conversions"] / df["clicks"].replace(0, np.nan)
    conv_col = "conversion_rate"

# Convert numeric
for c in [ctr_col, conv_col, "income", "age"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Remove negative incomes
df = df[df["income"] >= 0]

# USER SEGMENT BUCKETS
df["age_range"] = pd.cut(
    df["age"],
    bins=[15, 25, 35, 50, 65, 120],
    labels=["15-25", "25-35", "35-50", "50-65", "65+"],
    include_lowest=True
)

df["income_range"] = pd.cut(
    df["income"],
    bins=[0, 25000, 50000, 75000, 100000, float("inf")],
    labels=["0-25k", "25k-50k", "50k-75k", "75k-100k", "100k+"],
    include_lowest=True
)

df = df.dropna(subset=[ctr_col, conv_col])

# GROUPING: USER × AD combinations
group_cols = [
    "age_range", "gender", "income_range", "location",
    "ad_type", "ad_topic", "placement"
]

group_cols = [c for c in group_cols if c in df.columns]

agg = df.groupby(group_cols, observed=True).agg({
    ctr_col: "mean",
    conv_col: "mean",
}).reset_index()

agg = agg.rename(columns={ctr_col: "ctr", conv_col: "conversion_rate"})

# Normalize if needed
if agg["ctr"].max() > 1.1:
    agg["ctr"] /= 100.0
if agg["conversion_rate"].max() > 1.1:
    agg["conversion_rate"] /= 100.0

agg["ctr"] = agg["ctr"].fillna(0)
agg["conversion_rate"] = agg["conversion_rate"].fillna(0)

# Conversion per impression
agg["conv_per_impr"] = agg["ctr"] * agg["conversion_rate"]

# CATEGORY CAP LOGIC
num_groups = len(agg)

# Each category allowed up to ~3× its fair share
AVG_SHARE = TOTAL_BUDGET / num_groups
CAP = int(AVG_SHARE * 3)   # you can tweak multiplier here (e.g. 2x or 4x)

# But ensure it's not too small
CAP = max(CAP, 5000)

agg["ub"] = CAP

print(f"Number of groups = {num_groups}")
print(f"Per-category cap = {CAP}")

agg.to_csv("ads_agg.csv", index=False)
print("Saved aggregated data to ads_agg.csv (rows = {})".format(len(agg)))

Number of groups = 2705
Per-category cap = 5000
Saved aggregated data to ads_agg.csv (rows = 2705)
